# LMP2 Driver License Analysis

This notebook explores the history of the IMSA LMP2 class using the event data in this repository. We build the DuckDB database using the same SQL scripts referenced in the `Rakefile` and then break down results by driver license level.

In [ ]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='whitegrid')

## Build the DuckDB database

The database is created by executing `all-event-laps.sql` and `all-event-drivers.sql`. This mirrors the `rake db:update` task in the repository. The result is an in-memory DuckDB connection containing the `event_laps` and `event_drivers` tables.

In [ ]:
con = duckdb.connect()
with open('all-event-laps.sql') as f:
    con.execute(f.read())
with open('all-event-drivers.sql') as f:
    con.execute(f.read())

## Join lap data with driver information

For analysis we join the lap times with driver license information and focus on the LMP2 class.

In [ ]:
con.execute('''
CREATE OR REPLACE VIEW lmp2_laps AS
SELECT el.*, ed.license, ed.license_rank
FROM event_laps el
JOIN event_drivers ed
  ON el.year = ed.year
 AND el.event = ed.event
 AND el.session = ed.session
 AND el.driver_name = ed.name
WHERE el.class = 'LMP2';
''')

### Basic data overview

In [ ]:
overview = con.execute('''
SELECT MIN(year) AS first_year,
       MAX(year) AS last_year,
       COUNT(DISTINCT event) AS events,
       COUNT(DISTINCT session) AS sessions,
       COUNT(DISTINCT driver_name) AS drivers
FROM lmp2_laps;
''').df()
overview

## LMP2 drivers by license over the years

In [ ]:
license_year = con.execute('''
SELECT license,
       year,
       COUNT(DISTINCT session || '_' || car) AS entries,
       AVG(EXTRACT(EPOCH FROM lap_time)) AS avg_lap_seconds
FROM lmp2_laps
GROUP BY license, year
ORDER BY year, license;
''').df()
license_year.head()

In [ ]:
plt.figure(figsize=(10,6))
for license, grp in license_year.groupby('license'):
    plt.plot(grp['year'], grp['avg_lap_seconds'], marker='o', label=license)
plt.xlabel('Year')
plt.ylabel('Average Lap Time (s)')
plt.title('Average LMP2 Lap Time by Driver License')
plt.legend()
plt.show()

## Bronze drivers by circuit over time

This section focuses specifically on Bronze rated drivers. We compute their average lap time at every circuit for each year.

In [ ]:
bronze_event_year = con.execute('''
SELECT year, event,
       AVG(EXTRACT(EPOCH FROM lap_time)) AS avg_lap_seconds
FROM lmp2_laps
WHERE license = 'Bronze'
GROUP BY year, event
ORDER BY year, event;
''').df()
bronze_event_year.head()

In [ ]:
plt.figure(figsize=(12,6))
for event, grp in bronze_event_year.groupby('event'):
    plt.plot(grp['year'], grp['avg_lap_seconds'], marker='o', label=event)
plt.xlabel('Year')
plt.ylabel('Average Lap Time (s)')
plt.title('Bronze Driver Performance by Circuit')
plt.legend(bbox_to_anchor=(1.05,1), loc='upper left')
plt.tight_layout()
plt.show()

## License distribution in LMP2

How many unique drivers of each license level have competed in LMP2?

In [ ]:
license_counts = con.execute('''
SELECT license, COUNT(DISTINCT driver_name) AS drivers
FROM lmp2_laps
GROUP BY license
ORDER BY license_rank DESC;
''').df()
license_counts

In [ ]:
sns.barplot(data=license_counts, x='license', y='drivers')
plt.xlabel('License')
plt.ylabel('Unique Drivers')
plt.title('LMP2 Driver License Distribution')
plt.show()

## Future work

This notebook scratches the surface of what can be done with the IMSA dataset. Potential extensions include:

- Integrating results positions to study finishing trends.
- Analysing stint lengths and pit strategy by license level.
- Comparing qualifying vs race performance for different driver ratings.